# n = 5: Autoregressive covariance structure 

We try to parameterize covariance Gamma only using two parameters rho and sigma2 as in the AR(1) structure.

For n = 5 this autoregressive model converges in 15 iterations using Quasi-Newton, 13 with Newton. 

Here we just initialize beta under independent GLM assumptions, sigma2 using MM-Algorithm letting rho = 0, then MOM using the empirical covariance between Y_1 and Y_2.

We set IPOPT convergence tolerance to 10^-6, with the adaptive mean option turned on. 

$$\mu_i = 5, \rho = 0.9, \sigma^2 = 0.1, n_i = 5$$

$$\Gamma_i = \sigma^2 * \begin{pmatrix} 1 & \rho & \rho^2 & \rho^3 & \rho^4\\ \rho & 1 & \rho & \rho^2 & \rho^3 \\ \rho^2 & \rho & 1 & \rho & \rho^2 \\ \rho^3 & \rho^2 & \rho & 1 & \rho \\ \rho^4 & \rho^3 & \rho^2 & \rho & 1\end{pmatrix} \forall i \in [1, N = 10,000]$$ 

In [1]:
using DataFrames, Random, GLM, GLMCopula, Test
using LinearAlgebra, BenchmarkTools, Revise

Random.seed!(1234)

# sample size
N = 10_000
# observations per subject
n = 5
ρ = 0.1
σ2 = 0.1

V = zeros(n, n) # will store the AR(1) structure without sigma2

mean = 5

dist = Poisson

"""
    get_AR_cov(n, ρ, σ2, V)
Forms the AR(1) covariance structure given n (size of cluster), ρ (correlation parameter), σ2 (noise parameter)
"""
function get_AR_cov(n, ρ, σ2, V)
    @inbounds for i in 1:n
        V[i, i] = 1.0
        @inbounds for j in i+1:n
            V[i, j] = ρ^(j-i)
            V[j, i] = V[i, j]
        end
    end
    V
end

V = get_AR_cov(n, ρ, σ2, V)

# true Gamma
Γ = σ2 * V

5×5 Array{Float64,2}:
 0.1     0.01    0.001  0.0001  1.0e-5
 0.01    0.1     0.01   0.001   0.0001
 0.001   0.01    0.1    0.01    0.001
 0.0001  0.001   0.01   0.1     0.01
 1.0e-5  0.0001  0.001  0.01    0.1

In [2]:
vecd = [dist(mean) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [3.0, 8.0, 7.0, 4.0, 2.0]
 [2.0, 6.0, 6.0, 6.0, 7.0]
 [3.0, 5.0, 5.0, 10.0, 3.0]
 [5.0, 6.0, 3.0, 5.0, 2.0]
 [1.0, 10.0, 1.0, 8.0, 2.0]
 [5.0, 5.0, 4.0, 5.0, 4.0]
 [1.0, 4.0, 6.0, 5.0, 5.0]
 [7.0, 8.0, 6.0, 4.0, 2.0]
 [5.0, 3.0, 4.0, 6.0, 4.0]
 [5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 3.0, 6.0]
 [6.0, 3.0, 3.0, 8.0, 5.0]
 [3.0, 4.0, 7.0, 3.0, 4.0]
 ⋮
 [7.0, 4.0, 8.0, 5.0, 3.0]
 [2.0, 4.0, 4.0, 7.0, 7.0]
 [7.0, 2.0, 4.0, 6.0, 8.0]
 [7.0, 4.0, 8.0, 2.0, 6.0]
 [3.0, 6.0, 5.0, 5.0, 1.0]
 [3.0, 4.0, 4.0, 7.0, 7.0]
 [3.0, 4.0, 8.0, 2.0, 3.0]
 [2.0, 9.0, 7.0, 4.0, 6.0]
 [6.0, 4.0, 3.0, 4.0, 7.0]
 [6.0, 4.0, 4.0, 9.0, 6.0]
 [2.0, 3.0, 2.0, 5.0, 4.0]
 [4.0, 4.0, 2.0, 4.0, 4.0]

In [3]:
d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, N)

for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

gcm = GLMCopulaARModel(gcs);

In [4]:
initialize_model!(gcm)
@show gcm.β
@show exp.(gcm.β)
@show gcm.ρ
@show gcm.σ2;

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -112258.32152308253 39999
2 -112258.32152308253 -112258.32152308253 39999
gcm.β = [1.6199900423678915]
exp.(gcm.β) = [5.053040000000001]
gcm.ρ = [1.0]
gcm.σ2 = [0.08786419763584018]


In [5]:
Y_1 = [Y_Nsample[i][1] for i in 1:N]
Y_2 = [Y_Nsample[i][2] for i in 1:N]

update_rho!(gcm, Y_1, Y_2)
@show exp.(gcm.β)
@show gcm.ρ
@show gcm.σ2;

exp.(gcm.β) = [5.053040000000001]
gcm.ρ = [0.0022407940422173524]
gcm.σ2 = [0.08786419763584018]


In [6]:
loglikelihood!(gcm, true, true)

gcm1 = deepcopy(gcm)
gcm2 = deepcopy(gcm);

## Quasi Newton
## Number of Iterations....: 12
## 0.92 seconds 

In [7]:
@time GLMCopula.fit!(gcm1, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo", hessian_approximation = "limited-memory"))

gcm.θ = [1.6199900423678915, 0.0022407940422173524, 0.08786419763584018]
gcm.θ = [1.6199900423678915, 0.00999999, 0.08786419763584018]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        1
              

-112184.16665820027

# Newton
## Number of Iterations....: 14
## 0.84 seconds 

In [8]:
@time GLMCopula.fit!(gcm2, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",  hessian_approximation = "exact"))

gcm.θ = [1.6199900423678915, 0.0022407940422173524, 0.08786419763584018]
gcm.θ = [1.6199900423678915, 0.00999999, 0.08786419763584018]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequalit

-112184.16665820104

In [9]:
@show gcm1.∇θ
@show gcm2.∇θ;

gcm1.∇θ = [-7.583713719228058e-7, 3.432596518970765e-7, 4.908344464649161e-6]
gcm2.∇θ = [-9.010640802387115e-6, 1.9901345849060448e-9, 2.172579256498608e-6]


In [10]:
@show gcm1.θ
@show gcm2.θ;

gcm1.θ = [1.6120714434417753, 0.08241959732189377, 0.09896828294189804]
gcm2.θ = [1.6120714434634063, 0.08241959848405851, 0.09896828317475964]


In [11]:
@show loglikelihood!(gcm1, true, true)
@show loglikelihood!(gcm2, true, true);

loglikelihood!(gcm1, true, true) = -112184.16665820027
loglikelihood!(gcm2, true, true) = -112184.16665820104
